# In this Notebook, I use the pickled data in the previous notebook and build a Seq2Seq model using the API made by farizrahman4u

## I start with some utility cells that I always need in all my notebooks

In [1]:
# general modules for preprocessing and other stuff
import os # for os related calls
from subprocess import check_output

# for visualization stuff
import matplotlib.pyplot as plt
import numpy as np

# for pickling the data
from six.moves import cPickle as pickle

# for the seq2seq model
import keras
import seq2seq
from seq2seq.models import SimpleSeq2Seq

%matplotlib inline

Using TensorFlow backend.


In [2]:
def exec_command(cmd):
    '''
        function to execute a shell command and see it's 
        output in the python console
        @params
        cmd = the command to be executed along with the arguments
              ex: ['ls', '../input']
    '''
    print check_output(cmd).decode("utf8")

In [3]:
''' define the important paths for the Task '''
data_path = "../Data/cornell_movie_dialogs_corpus"
train_file = os.path.join(data_path, "movie_lines.txt")
save_path = os.path.join(data_path, "Data_final.pickle")
model_path = "../Models"

''' define a few important constant --variables. '''
field_separator = "+++$+++"

# Ok! Now let's build the Model

In [4]:
# Set the hyper parameters:
input_length = 35
input_dim = 64
hidden_dim = 10
output_dim = input_dim
output_length = input_length
depth = 3 # 3 for encoder and 3 for decoder. So 6 in total.

In [5]:
# The model for seq2seq (ChatBot)

'''
*****************************************************************************************************************
The Model used for training on the data:
*****************************************************************************************************************
'''
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding

main_model = SimpleSeq2Seq(output_length, 1, input_dim=input_length, hidden_dim=hidden_dim, depth=depth)
main_model.compile(loss='mse', optimizer='rmsprop')

# add_model = Sequential()
# add_model.add(Embedding(len(dictionary), 64, input_length=input_length))

# model = Model(inputs=add_model.input, outputs=main_model(add_model.output))
main_model.compile('rmsprop', 'mse')

In [6]:
main_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, None, 35)      0                                            
____________________________________________________________________________________________________
private__optional_input_place_ho (2,)                  0                                            
____________________________________________________________________________________________________
private__optional_input_place_ho (2,)                  0                                            
____________________________________________________________________________________________________
private__optional_input_place_ho (2,)                  0                                            
___________________________________________________________________________________________

In [7]:
# get the data from the pickled dictionary
with open(save_path, "rb") as pickle_file:
    my_dict = pickle.load(pickle_file)
    
len(my_dict)

3

In [8]:
# extract the data from the pickle
Data = my_dict['data'] 
dictionary = my_dict['mapping']
reverse_dictionary = my_dict['rev_mapping']

In [9]:
Data = Data.reshape((Data.shape[0], 1, Data.shape[1]))
print Data.shape
Data[: 3]

(286730, 1, 35)


array([[[   110,     25,   2325, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642]],

       [[   110,     25,   4074, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642]],

       [[     0,    434,    445, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642, 164642, 164642, 164642, 164642, 164642,
         164642, 164642, 164642]]], dtype=int32)

In [10]:
# Create the Fitting data
data_X = Data[:len(Data) - 1]
data_y = Data[1:]
# check if the data lengths are equal: 
print len(Data), len(data_X), len(data_y)

286730 286729 286729


In [ ]:
# running for only one epoch for now.
path = os.path.join(model_path, "Model1")
if(os.path.isfile(path)):
    main_model.load_weights(path)
    print "Earlier Model loaded"
    
main_model.fit(data_X, data_y, epochs=5)

Earlier Model loaded
Epoch 1/5
286729/286729 [==============================] - 80s - loss: 4339623282.6884    
Epoch 2/5
286729/286729 [==============================] - 82s - loss: 4444212526.9325    
Epoch 3/5
286729/286729 [==============================] - 85s - loss: 4743843835.2466    
Epoch 4/5
286729/286729 [==============================] - 79s - loss: 5814930786.8139    
Epoch 5/5
212000/286729 [=====================>........] - ETA: 22s - loss: 8684888459.6504

# OMG!! Can't believe the loss figure!

In [12]:
# save the model here:
main_model.save_weights(os.path.join(model_path, "Model1"))

In [13]:
# Random test this model for once:
input_sentence = "Hello! I am a programmer.".split()

# convert the input sentence using the dictionary:
input_data = np.array(input_sentence + ([" "] * (input_length - len(input_sentence))))
input_data

array(['Hello!', 'I', 'am', 'a', 'programmer.', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
      dtype='|S11')

In [14]:
for i in range(len(input_data)):
    try:
        input_data[i] = dictionary[input_data[i]]
    
    except KeyError: 
        input_data[i] = dictionary["other"]

In [15]:
input_data = input_data.reshape((1, 1, input_length))

In [16]:
output_data = main_model.predict(input_data)

In [17]:
reduce(lambda x, y: x + " " + y, map(lambda x: reverse_dictionary[x], map(int, output_data[0, 0, :])))

'religious fun... Casablanca, functional. saltier polyester. Cy. life.</i> bellied\x97 sparrowlike Partisans patsy". Why". Dunois? ground... Assets Four-Eyes\'ll Decker.... so...convincing. playing...we forgetful, wiretap, Marathon. Senate--uh--what home\'s Tiberius reefer. Equitation. Glaucoma? oaf withal, Nigger\'s Shaky? tch, dentist\'s?'

# Currently, It is returning some garbled senseless Replies. But, If we train it for longer, tweak some parameters, the result should improve.